In [65]:
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm.notebook import tqdm

In [155]:
#data_prefix = "ling_data/"
data_prefix = "data/"
course = pd.read_csv(data_prefix + "course.csv")
course_element = pd.read_csv(data_prefix + "course_element.csv") 
course_module = pd.read_csv(data_prefix + "course_module.csv") 
sol_log = pd.read_csv(data_prefix + "solution_log.csv") 
u_course_progress = pd.read_csv(data_prefix + "user_course_progress.csv") 
u_element_progress = pd.read_csv(data_prefix + "user_element_progress.csv") 
u_module_progress = pd.read_csv(data_prefix + "user_module_progress.csv")
users = pd.read_csv("user_course_ids.csv")

sol_log.submission_time = pd.to_datetime(sol_log.submission_time)
u_element_progress.time_updated = pd.to_datetime(u_element_progress.time_updated)
u_module_progress.time_unlocked = pd.to_datetime(u_module_progress.time_unlocked)
u_element_progress.time_achieved = pd.to_datetime(u_element_progress.time_achieved)

C:\Users\Sirius\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Sirius\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [177]:
def calc_active_days_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log):
    sub_time = u_element_progress.merge(sol_log, left_on='id', right_on='element_progress_id')[['user_id', 'submission_time', 'course_id']]
    sub_time = sub_time[(sub_time.submission_time < date) & (sub_time.submission_time > date - timedelta(days=window_size))]
    sub_time.submission_time = sub_time.submission_time.dt.round("D")
    d = sub_time.groupby(["user_id", "course_id"]).submission_time.nunique()
    u_course_progress["active_days"] = u_course_progress.merge(d, on=['user_id', 'course_id'], how="left").submission_time.fillna(0)
    
    
    result = pd.DataFrame()
    result["user_id"] = u_course_progress.user_id
    result["course_id"] = u_course_progress.course_id
    result["active_days"] = u_course_progress.active_days / window_size
    return result


## local solving speed (solved problems in periods divided by active days in )
def calc_solving_speed_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log):
    
    active_days = calc_active_days_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log)
    
    calc_progress_current_for_users = 

## user progress in (date - window_size days, date)
def calc_progress_current_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log):
    d = u_element_progress[(u_element_progress.achieve_reason == "solved") & (u_element_progress.course_element_type == "task") &
                   (u_element_progress.time_updated < date) & (u_element_progress.time_updated > date - timedelta(days=window_size))].groupby(["user_id", "course_id"]).progress_current.sum()
    result = pd.DataFrame()
    result["user_id"] = u_course_progress.user_id
    result["course_id"] = u_course_progress.course_id
    result["progress_current"] = result.merge(d, on=["user_id", "course_id"], how="left").progress_current.fillna(0)
    return result

    
def calc_task_completeness_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log):
    module2totalpoints = course_element[course_element.element_type == "task"].groupby("module_id").score.sum()
    
    module2tasks = dict(course_element[course_element.element_type == "task"].groupby("module_id").id.count())
    u_module_progress["task_count"] = u_module_progress.course_module_id.map(module2tasks)

    uidcid2possible_tasks = u_module_progress[u_module_progress.time_unlocked < date].groupby(["user_id", "course_id"]).task_count.sum()
    uidcid2solved_before = u_element_progress[(u_element_progress.time_achieved <= date - timedelta(days=window_size)) & (u_element_progress.course_element_type == "task")].groupby(["user_id", "course_id"]).progress_current.sum()
    period_progress = calc_progress_current_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                                  u_element_progress, sol_log)


    uidcid2solved_before.name = "solved_before"
    uidcid2possible_tasks.name = "possible_tasks"
    
    period_progress = period_progress.merge(uidcid2solved_before, on=["user_id", "course_id"], how="left").fillna(0).merge(uidcid2possible_tasks, on=["user_id", "course_id"], how="left").dropna()
    result = pd.DataFrame()
    result["user_id"] = period_progress.user_id
    result["course_id"] = period_progress.course_id
    result["task_completeness"] = period_progress.progress_current / (period_progress.possible_tasks - period_progress.solved_before)
    return result


In [61]:
ans1 = calc_active_days_for_users(date, window_size, course, course_module, course_element, u_course_progress, u_module_progress, 
                              u_element_progress, sol_log)

Фичи: 
course_done - получен ли сертификат 1 уровня
progress_current_ratio - доля полученных баллов за курс
progress_current - сколько всего баллов полученно 
total_courses_done - сколько всего закрыл курсов ученик
total_tries_overall - сколько попыток сдачи сделал на всех курсах 
tries_count - сколько попыток сделал ученик на данном курсе
task_problems_in_course	text_problems_in_course	video_problems_in_course - сколько в курсе соответсвующих задач
mean_progress - на сколько баллов люди в среднем прорешивают курс
total_participants - сколько всего участников было на курсе
advanced_modules_count - сколько в курсе продвинутых модулей (в геометрии 0)
transferred_count - сколько задач пользователем перенесено
tries_efficiency - кол-во баллов в курсе делить на кол-во попыток в курсе
active_days - кол-во активных дней работы

То что еще надо юзать: 
knowledge_area_id

In [118]:
u_course_progress = u_course_progress.merge(users, on=['user_id', 'course_id'])

In [119]:
## Посчитаем, сколько в курсе можно заработать максимум баллов
course_to_max_points = dict(course_module.groupby("course_id").progress_max.sum())


## Подсчитаем, закрыл ли пользователь курс на сертификат первого уровня

d = dict(course_module[~course_module.is_advanced & course_module.level == 1].groupby(by="course_id").id.unique().apply(len))
u_course_progress["course_done"] = u_course_progress.course_id.map(d) <= u_course_progress.modules_achieved


## Подсчитаем по пользователю, сколько баллов он заработал

u_course_progress = u_course_progress.merge(u_element_progress.groupby(['user_id', 'course_id']).\
                                            progress_current.sum(), on=['user_id', 'course_id'])

## Найдем долю баллов которую получил пользователь от максимального числа баллов

u_course_progress["progress_current_ratio"] = u_course_progress.progress_current / u_course_progress.course_id.map(course_to_max_points)


## Найдем, сколько всего ученик закрыл курсов

u_course_progress["total_courses_done"] = u_course_progress.user_id.map(dict(u_course_progress.groupby("user_id").course_done.sum()))





## Найдем, сколько всего попыток сдачи задач, сделал ученик. total_tries_overall - считая все курсы, 
## total_tries - учитывая только данный курс

u_course_progress["total_tries_overall"] = u_course_progress.user_id.map(dict(u_element_progress.groupby("user_id").tries_count.sum()))
u_course_progress = u_course_progress.merge( u_element_progress.groupby(["user_id", "course_id"]) \
                        .tries_count.sum(), on=['user_id', 'course_id'], how = "left")





## Найдем, сколько в курсе видео / текстовых / задачный элементов

course_element["course_id"] = course_element.module_id.map(dict(zip(course_module.id, course_module.course_id)))

for name in ["task", "text", "video"]:
    u_course_progress[name + "_problems_in_course"] = u_course_progress.course_id. \
    map(dict(course_element[course_element.element_type == name].groupby("course_id").id.count()))
    
elem_count = u_course_progress["task_problems_in_course"] + u_course_progress["video_problems_in_course"] + u_course_progress["text_problems_in_course"]

for name in ["task", "text", "video"]:
    u_course_progress[name + "_problems_in_course"] /= elem_count
     


## Найдем, сколько баллов получают люди за курс в среднем

u_course_progress["mean_progress"] = u_course_progress.course_id.map(dict(u_course_progress.groupby("course_id").progress_current_ratio.sum() / u_course_progress.groupby("course_id").progress_current_ratio.count()))


## Найдем, сколько всего участников на курсе


u_course_progress["total_participants"] = u_course_progress.course_id.map(dict(u_course_progress.groupby("course_id").id.count()))



## Найдем, сколько продвинутых модулей на курсе
u_course_progress["advanced_modules_count"] = u_course_progress.course_id.map(course_module.groupby("course_id").is_advanced.sum())



## Посчитаем, сколько задач каждый пользователь перенес 

d = u_element_progress[u_element_progress.achieve_reason == "transferred"].groupby(["user_id", "course_id"]).id.count()
u_course_progress["transferred_count"] = u_course_progress.merge(d, on=['user_id', 'course_id'], how="left").id_y.fillna(0)


## Посчитаем кол-во баллов полученное за курс учеников разделенное на количество попыток в курсе
u_course_progress["tries_efficiency"] = u_course_progress.progress_current / u_course_progress.tries_count


## Посчитаем  дни активности

sub_time = u_element_progress.merge(sol_log, left_on='id', right_on='element_progress_id')[['user_id', 'submission_time', 'course_id']]

sub_time.submission_time = pd.to_datetime(sub_time.submission_time)
sub_time.submission_time = sub_time.submission_time.dt.round("D")

d = sub_time.groupby(["user_id", "course_id"]).submission_time.nunique()

u_course_progress["active_days"] = u_course_progress.merge(d, on=['user_id', 'course_id'], how="left").submission_time.fillna(0)


## Найдем баллы деленные на кол-во дней активности (скорость набора баллов)
u_course_progress["solving_speed"] = u_course_progress.progress_current / u_course_progress.active_days


## я не знаю что это и где оно появляется
u_course_progress.drop(columns='Unnamed: 0', inplace=True)


u_course_progress.to_csv("to_combine/user_course_progress_done.csv", index=None)


In [24]:
from datetime import timedelta
import datetime

In [15]:
window_size = 14

In [80]:
date = pd.to_datetime('2020-07-30') 

In [4]:
sub_time = u_element_progress.merge(sol_log, left_on='id', right_on='element_progress_id')[['user_id', 'submission_time', 'course_id']]

sub_time.submission_time = pd.to_datetime(sub_time.submission_time)
sub_time.submission_time = sub_time.submission_time.dt.round("D")


d = sub_time.groupby(["user_id", "course_id"]).submission_time.nunique()

u_course_progress["active_days"] = u_course_progress.merge(d, on=['user_id', 'course_id'], how="left").submission_time.fillna(0)